## Step -1 Business Problem Understanding

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Step -2 Data Understanding

#### Load data & Understand every variable

In [2]:
df = pd.read_csv('insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


#### dataset understanding

In [3]:
df.shape

(1338, 7)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   expenses  1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [5]:
df['sex'].value_counts()

sex
male      676
female    662
Name: count, dtype: int64

In [6]:
df['children'].value_counts()

children
0    574
1    324
2    240
3    157
4     25
5     18
Name: count, dtype: int64

In [7]:
df['smoker'].value_counts()

smoker
no     1064
yes     274
Name: count, dtype: int64

In [8]:
df['region'].value_counts()

region
southeast    364
southwest    325
northwest    325
northeast    324
Name: count, dtype: int64

#### Exploratory Data Analysis

In [9]:
continuous_features = ['age', 'bmi', 'expenses']
discrete_categorical = ['sex', 'smoker', 'region']
discrete_count = ['children']

In [10]:
df[continuous_features].describe()

,age,bmi,expenses
count,1338.000000,1338.000000,1338.000000
mean,39.207025,30.665471,13270.422414
std,14.049960,6.098382,12110.011240
min,18.000000,16.000000,1121.870000
25%,27.000000,26.300000,4740.287500
50%,39.000000,30.400000,9382.030000
75%,51.000000,34.700000,16639.915000
max,64.000000,53.100000,63770.430000


In [11]:
df[discrete_categorical].describe()

,sex,smoker,region
count,1338,1338,1338
unique,2,2,4
top,male,no,southeast
freq,676,1064,364


In [12]:
df[continuous_features].corr()

,age,bmi,expenses
age,1.000000,0.109341,0.299008
bmi,0.109341,1.000000,0.198576
expenses,0.299008,0.198576,1.000000


## Step - 3: Data Preprocessing

#### Data Cleaning

In [13]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
expenses    0
dtype: int64

In [14]:
df.duplicated().sum()

1

In [15]:
df.drop_duplicates(inplace=True)

In [16]:
df.shape

(1337, 7)

In [17]:
df.drop('region', axis=1, inplace=True)

#### Encoding

In [18]:
# encoding sex column
df['sex'].replace({'female':0, 'male':1}, inplace=True)

# encoding 'smoker' column
df['smoker'].replace({'no':0, 'yes':1}, inplace=True)

#### X&y

In [19]:
X = df.drop('expenses', axis=1)
y = df['expenses']

## Step - 4: Modeling

### findout the best random_state value 

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

Train = []
Test = []
CV = []

for i in range(0, 100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
    model = LinearRegression()
    model.fit(X_train, y_train)

    ypred_train = model.predict(X_train)
    ypred_test = model.predict(X_test)

    Train.append(model.score(X_train, y_train))
    # print('Train R2:', r2_score(y_train, ypred_train))
    Test.append(model.score(X_test, y_test))
    # print('Test R2:', r2_score(y_test, ypred_test))

    CV.append(cross_val_score(model, X_train, y_train, cv=5).mean())


em = pd.DataFrame({'Train':Train, 'Test':Test, 'CV':CV})
gm = em[(abs(em['Train']-em['Test']) <= 0.05) & (abs(em['Test']-em['CV']) <=0.05)]
print('best random state number:', gm[gm['Test']==gm['Test'].max()].index.to_list())

best random state number: [90]


In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=90)

# <font color = aqua> Polynomial Regresion </font> 

## Hyperparameter Tuning
#### Choosing the best polynomial degree for given dataset

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures


train_r2 = []
test_r2 = []
for i in range(1, 10):
    # data preprocessing on train data
    polynomial_converter = PolynomialFeatures(degree=i)
    X_train_poly = pd.DataFrame(polynomial_converter.fit_transform(X_train))

    # modelling on train data
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(X_train_poly, y_train)


    # prediction and evaluation on train data
    ypred_train = model.predict(X_train_poly)
       # print('Train R2:', i, model.score(X_train_poly, y_train))
    train_r2.append(model.score(X_train_poly, y_train))

    # transformation on test data
    X_test_poly = pd.DataFrame(polynomial_converter.fit_transform(X_test))

    # prediction and evaluation on test data
    ypred_test = model.predict(X_test_poly)
       # print('Test R2:', i, model.score(X_test_poly, y_test))
    test_r2.append(model.score(X_test_poly, y_test))

In [23]:
train_r2

[0.7408889895315596,
 0.8331314951008288,
 0.8315194582650793,
 0.8512842583640126,
 0.8638455011376718,
 0.8769764871061212,
 0.8875930315161533,
 0.8639136749635385,
 0.9089294816940532]

In [24]:
test_r2

[0.7814830789571359,
 0.883076067634921,
 0.8461044557399258,
 0.8112069152864396,
 0.7377295753644332,
 -2.4590264327865805,
 -377.2238364833275,
 -7155.643907267677,
 -15925.494774260704]

##### between both train_r2 and test_r2, we can see index number 4 is having highest degree

### Build a model

In [33]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

polynomial_converter = PolynomialFeatures(degree=4, include_bias=True)
X_train_poly = pd.DataFrame(polynomial_converter.fit_transform(X_train))
X_test_poly = pd.DataFrame(polynomial_converter.fit_transform(X_test))

model = LinearRegression()
model.fit(X_train_poly, y_train)
print('Intercept:', model.intercept_)
#print('Coefficients:', model.coef_)

ypred_train = model.predict(X_train_poly)
ypred_test = model.predict(X_test_poly)

print('Train R2:', r2_score(y_train, ypred_train))
print('Test R2:', r2_score(y_test, ypred_test))
from sklearn.model_selection import cross_val_score
print('Cross Validation Score:', cross_val_score(model, X_train_poly, y_train, cv=5).mean())

Intercept: 921.6090469079099
Train R2: 0.8512842583640126
Test R2: 0.8112069152864396
Cross Validation Score: 0.8126643086646149


##### on the future data, we are also needed to transform the test data with polynomial 

# Predictions on a new data

In [26]:
test_df = {'age':35,
             'sex':'Male',
             'bmi':31.4,
             'children':5,
             'smoker':'yes',
             'region':'southeast'}
df_test = pd.DataFrame(test_df, index=[0])
df_test

,age,sex,bmi,children,smoker,region
0,35,Male,31.4,5,yes,southeast


In [27]:
df_test.drop('region', axis=1, inplace=True)

In [28]:
df_test['sex'].replace({'female':0, 'Male':1}, inplace=True)
df_test['smoker'].replace({'no':0, 'yes':1}, inplace=True)
X2 = df_test
X2

,age,sex,bmi,children,smoker
0,35,1,31.4,5,1


In [29]:
transformed_data = polynomial_converter.fit_transform(X2)
transformed_data

array([[1.00000000e+00, 3.50000000e+01, 1.00000000e+00, 3.14000000e+01,
        5.00000000e+00, 1.00000000e+00, 1.22500000e+03, 3.50000000e+01,
        1.09900000e+03, 1.75000000e+02, 3.50000000e+01, 1.00000000e+00,
        3.14000000e+01, 5.00000000e+00, 1.00000000e+00, 9.85960000e+02,
        1.57000000e+02, 3.14000000e+01, 2.50000000e+01, 5.00000000e+00,
        1.00000000e+00, 4.28750000e+04, 1.22500000e+03, 3.84650000e+04,
        6.12500000e+03, 1.22500000e+03, 3.50000000e+01, 1.09900000e+03,
        1.75000000e+02, 3.50000000e+01, 3.45086000e+04, 5.49500000e+03,
        1.09900000e+03, 8.75000000e+02, 1.75000000e+02, 3.50000000e+01,
        1.00000000e+00, 3.14000000e+01, 5.00000000e+00, 1.00000000e+00,
        9.85960000e+02, 1.57000000e+02, 3.14000000e+01, 2.50000000e+01,
        5.00000000e+00, 1.00000000e+00, 3.09591440e+04, 4.92980000e+03,
        9.85960000e+02, 7.85000000e+02, 1.57000000e+02, 3.14000000e+01,
        1.25000000e+02, 2.50000000e+01, 5.00000000e+00, 1.000000

In [30]:
model.predict(transformed_data)

array([26684.35830637])